In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [ ]:
import os
import cv2
import numpy as np
import json
import pandas as pd
from datetime import datetime

def merge_and_count_dots(image_path_dots, mask_path_nucleus):
    """
    Merges a green-dot image with a binary nucleus mask and processes all dots,
    indicating whether each is inside or outside the nucleus.
    """
    # Load green dot image (assume dots on black background)
    image_dots = cv2.imread(image_path_dots, cv2.IMREAD_UNCHANGED)
    if image_dots is None:
        return f"Error loading dot image: {image_path_dots}", None, 0, 0
    
    # Convert dots image to 3-channel if single-channel
    if len(image_dots.shape) == 2:
        image_dots = cv2.cvtColor(image_dots, cv2.COLOR_GRAY2BGR)
    
    # Load binary nucleus mask (0 for background, 255 for nucleus)
    mask_nucleus = cv2.imread(mask_path_nucleus, cv2.IMREAD_GRAYSCALE)
    if mask_nucleus is None:
        return f"Error loading nucleus mask: {mask_path_nucleus}", None, 0, 0
    
    # Ensure mask has same dimensions as image
    mask_nucleus = cv2.resize(mask_nucleus, (image_dots.shape[1], image_dots.shape[0]))
    
    # Threshold the dots image to isolate green dots
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([80, 255, 255])
    hsv_image_dots = cv2.cvtColor(image_dots, cv2.COLOR_BGR2HSV)
    green_mask = cv2.inRange(hsv_image_dots, lower_green, upper_green)
    
    # Find contours for dots
    contours, _ = cv2.findContours(green_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Process all dots
    dot_info = []
    count_inside_nucleus = 0
    count_outside_nucleus = 0
    
    # Create a copy for visualization
    annotated_image = image_dots.copy()
    
    for i, contour in enumerate(contours):
        x, y, w, h = cv2.boundingRect(contour)
        cx, cy = x + w // 2, y + h // 2  # Center of the dot

        # Check if dot is inside or outside nucleus
        is_inside_nucleus = mask_nucleus[cy, cx] > 0
        
        # Update counts and set color
        if is_inside_nucleus:
            count_inside_nucleus += 1
            box_color = (0, 0, 255)  # Green box for dots inside nucleus
        else:
            count_outside_nucleus += 1
            box_color = (255, 0, 0)  # Blue box for dots outside nucleus
            
        # Store dot information
        dot_info.append({
            'dot_id': i + 1,
            'position': {'x': cx, 'y': cy},
            'width': w,
            'height': h,
            'area': int(cv2.contourArea(contour)),
            'location': 'inside_nucleus' if is_inside_nucleus else 'outside_nucleus'
        })
        
        # Draw only the bounding box, no text labels
        cv2.rectangle(annotated_image, (x, y), (x + w, y + h), box_color, 2)

    # Create visualization
    mask_colored = cv2.cvtColor(mask_nucleus, cv2.COLOR_GRAY2BGR)
    mask_colored = cv2.addWeighted(mask_colored, 0.3, np.full_like(mask_colored, 255), 0, 0)
    merged_image = cv2.addWeighted(annotated_image, 0.7, mask_colored, 0.3, 0)
    
    return merged_image, dot_info, count_inside_nucleus, count_outside_nucleus

def get_matching_mask_file(pml_file, mask_folder):
    """
    Get the corresponding mask file for a PML body image file.
    Example: 
    PML file: flattened_position_2-1_C1.tif
    Mask file: flattened_position_2-1_C0_mask_mask.tif
    """
    # Extract position number from PML file (e.g., "2-1" from "flattened_position_2-1_C1.tif")
    position = pml_file.split('_')[2]
    mask_filename = f"ha_flattened_position_{position}_C0.tif"
    mask_path = os.path.join(mask_folder, mask_filename)
    return mask_path

def process_folder(input_folder, mask_folder, output_folder):
    """
    Process all images in a folder and save results.
    """
    os.makedirs(output_folder, exist_ok=True)
    
    # Get list of PML body image files (ending with C1.tif)
    image_files = [f for f in os.listdir(input_folder) 
                  if f.endswith('C1.tif') and 'ha_flattened_position' in f]
    
    # Initialize results dictionary
    results_data = {
        'image_results': []
    }
    
    for image_file in image_files:
        print(f"Processing {image_file}...")
        
        # Construct full paths
        image_path = os.path.join(input_folder, image_file)
        mask_path = get_matching_mask_file(image_file, mask_folder)
        
        # Check if mask file exists
        if not os.path.exists(mask_path):
            print(f"Warning: No matching mask file found for {image_file}")
            results_data['metadata']['failed_processes'] += 1
            results_data['image_results'].append({
                'filename': image_file,
                'status': 'failed',
                'error': 'No matching mask file found'
            })
            continue
            
        # Process image
        merged_image, dot_info, count_inside, count_outside = merge_and_count_dots(image_path, mask_path)
        
        if isinstance(merged_image, str):  # Error occurred
            print(f"Error processing {image_file}: {merged_image}")
            results_data['metadata']['failed_processes'] += 1
            results_data['image_results'].append({
                'filename': image_file,
                'status': 'failed',
                'error': merged_image
            })
            continue
            
        # Save merged image
        output_path = os.path.join(output_folder, f"ha_processed_{image_file}")
        cv2.imwrite(output_path, merged_image)
        
        # Store results for this image
        image_result = {
            'filename': image_file,
            'status': 'success',
            'mask_filename': os.path.basename(mask_path),
            'total_dots': len(dot_info),
            'dots_inside_nucleus': count_inside,
            'dots_outside_nucleus': count_outside,
            'dot_details': dot_info,
            'output_image': f"ha_processed_{image_file}"
        }
        results_data['image_results'].append(image_result)
        
        # Update summary statistics
        
    
    # Update final metadata and summary statistics
    
    
    # Save all results to a single JSON file
    json_path = os.path.join(output_folder, 'high_arsenic_annotations.json')
    with open(json_path, 'w') as f:
        json.dump(results_data, f, indent=4)
    
    # Save summary results as CSV for quick reference
    # if results_data['metadata']['successful_processes'] > 0:
    #     summary_df = pd.DataFrame([{
    #         'filename': r['filename'],
    #         'status': r['status'],
    #         'total_dots': r.get('total_dots', 0),
    #         'dots_inside_nucleus': r.get('dots_inside_nucleus', 0),
    #         'dots_outside_nucleus': r.get('dots_outside_nucleus', 0)
    #     } for r in results_data['image_results']])
        
    #     csv_path = os.path.join(output_folder, 'summary_results.csv')
    #     summary_df.to_csv(csv_path, index=False)
    
    return results_data

# Example usage
if __name__ == "__main__":
    input_folder = "E:/MLOps/Project/input/pml_high_arsenic_input"  # Folder containing PML body images
    mask_folder = "E:/MLOps/Project/input/binary_mask_nucleus_high_arsenic_input"  # Folder containing nucleus mask images
    output_folder = "E:/MLOps/Project/inputs/High_Arsenic_masks"
    
    results = process_folder(input_folder, mask_folder, output_folder)

Processing flattened_position_10_C1.tif...
Processing flattened_position_11_C1.tif...
Processing flattened_position_12_C1.tif...
Processing flattened_position_13_C1.tif...
Processing flattened_position_14_C1.tif...
Processing flattened_position_15_C1.tif...
Processing flattened_position_16_C1.tif...
Processing flattened_position_17_C1.tif...
Processing flattened_position_18_C1.tif...
Processing flattened_position_19_C1.tif...
Processing flattened_position_1_C1.tif...
Processing flattened_position_20_C1.tif...
Processing flattened_position_21_C1.tif...
Processing flattened_position_22_C1.tif...
Processing flattened_position_23_C1.tif...
Processing flattened_position_24_C1.tif...
Processing flattened_position_25_C1.tif...
Processing flattened_position_26_C1.tif...
Processing flattened_position_27_C1.tif...
Processing flattened_position_28_C1.tif...
Processing flattened_position_29_C1.tif...
Processing flattened_position_2_C1.tif...
Processing flattened_position_30_C1.tif...
Processing fl